In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time


In [541]:
driver = webdriver.Chrome()
url = "https://www.pvrcinemas.com/Bengaluru/cinemas/showtimes"
movie_name = "JAWAN"
city = "Bengaluru"
driver.get(url)

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "movie-box")))

movie_elements = driver.find_elements(By.CLASS_NAME, "movie-box")
for movie_element in movie_elements:
    movie_title = movie_element.find_element(By.CLASS_NAME, "m-title")
    if movie_title and movie_name in movie_title.text:

        booking_button = movie_element.find_element(By.CSS_SELECTOR, ".btn.btn-primary-white.text-uppercase.ng-star-inserted")
        driver.get(booking_button.get_attribute("href"))
        break

In [542]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
time.sleep(3)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
driver.execute_script("window.scrollTo(0, 0);")
len(cinema_list)


23

In [545]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
driver.execute_script("window.scrollTo(0, 0);")
print(len(cinema_list))

show_times=dict()

for i in range(3):
    cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
    cinema = cinema_list[i]
    print(cinema.find_element(By.CLASS_NAME, "title").text)
    cinema_button = cinema.find_element(By.CLASS_NAME, "cinema-title")
    cinema_button.click()
    hindi_2d_elements = cinema.find_elements(By.XPATH, '//h4[contains(@class, "type-title") and contains(text(), "HINDI- 2D")]')
    for hindi_2d_element in hindi_2d_elements:
        if len(hindi_2d_element.text)==0:
            continue
        ul_element = hindi_2d_element.find_element(By.XPATH, './following-sibling::ul[@class="type-time-slots ng-star-inserted"]')
        slot_elements = ul_element.find_elements(By.XPATH, './/li[@class="ng-star-inserted"]') 
        for slot in slot_elements:
            n=len(show_times)
            if (len(slot.text.strip()) != 0) and (slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1] == "success"):
                show_times[n] = {
                    "name" : cinema.find_element(By.CLASS_NAME, "title").text,
                    "time" : slot.text.strip(),
                    "type" : slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1]
                } 
                break
        break

    try:
        slot.click()
        time.sleep(3)
        wait = WebDriverWait(driver, 5)
        loaded = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "skip-btn")))

        skip_btn = driver.find_element(By.CLASS_NAME, "skip-btn")
        skip_btn.click()

    except:
        print("no offers")

    time.sleep(5)
    wait = WebDriverWait(driver, 5)
    loaded = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "modal-content-header")))

    close_class = driver.find_element(By.CLASS_NAME, "modal-content-header")
    btn = close_class.find_element(By.CLASS_NAME, "ion-android-close")
    btn.click()

    seat_prices_array = [title.find_element(By.XPATH, ".//span").text for title in driver.find_elements(By.CLASS_NAME, "seats-col") if title.find_element(By.XPATH, ".//span").get_attribute("class")=="area hshshs"]
    seat_prices = [int(float(re.search(r'(\d+\.\d+)', seat).group(1))) for seat in seat_prices_array]

    n_seats = 0
    n_seats_available = 0
    rows = driver.find_elements(By.CSS_SELECTOR, ".seats-row.ng-star-inserted")
    for row in rows:
        n_seats+= len(row.find_elements(By.CSS_SELECTOR, ".seats-col.ng-star-inserted"))
        n_seats_available += len([1 for x in row.find_elements(By.XPATH, './/span') if x.get_attribute("class") == 'seat current'])
    
    show_times[n]["total seats"] = n_seats
    show_times[n]["seats available"] = n_seats_available    
    show_times[n]["Pricing"] = '/'.join(map(str, seat_prices))

    btn = driver.find_element(By.CLASS_NAME, "ion-arrow-left-c")
    btn.click()
    wait = WebDriverWait(driver, 5)
    element_on_next_page = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")))

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
    driver.execute_script("window.scrollTo(0, 0);")

17
PVR VEGA Bengaluru
PVR Nexus (Formerly Forum), Koramangala
INOX Central, Mantri Junction, JP Nagar
no offers


In [4]:
from functions import * 

theatres = create_cinemas_list("Bengaluru")
driver = webdriver.Chrome()
open_booking_page("JAWAN", driver)
shows = create_show_list(driver)

0 23 INOX Garuda Yelahanka
no offers
1 0 PVR Nexus (Formerly Forum), Koramangala
2 1 INOX Central, Mantri Junction, JP Nagar
no offers
error


In [6]:
final_df = shows.merge(theatres, how="left", on="name")

In [52]:
import pandas as pd
import sqlite3
import os
basedir = os.getcwd()
parent_dict = os.path.dirname(basedir)
db_path = os.path.join(parent_dict, 'todo.db')
conn = sqlite3.connect(db_path)
final_df.to_sql('movie_shows', conn, if_exists='replace', index=True)
conn.commit()
conn.close()


In [51]:
final_df["Longitude"] = final_df["Longitude"].astype(float)
final_df["Latitude"] = final_df["Latitude"].astype(float)
final_df

,Theatre,Time,type,TotalSeats,SeatsAvailable,Cost,Latitude,Longitude,Movie,City
0,INOX Garuda Yelahanka,9:30 PM,success,216,105,270/360,13.097762,77.581719,JAWAN,Bengaluru
1,PVR VEGA Bengaluru,9:30 PM,warning,368,50,300/320/580,12.915512,77.613051,JAWAN,Bengaluru
2,"PVR Nexus (Formerly Forum), Koramangala",8:50 PM,warning,NaN,NaN,NaN,12.934801,77.611585,JAWAN,Bengaluru


In [53]:
final_df["Latitude"]

0    13.097762
1    12.915512
2    12.934801
Name: Latitude, dtype: float64